In [1]:
# Import numpy for random number generation
import numpy as np
from random import randbytes
# importing Qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
# Import basic plotting tools
from helpers import *
from qiskit.visualization import plot_histogram
from dotenv import load_dotenv
import os

load_dotenv()
provider = QiskitRuntimeService(token=os.environ["ibm_token"], channel="ibm_quantum")

# Selecting a backend
real_backend = provider.backend("ibm_brisbane")
backend = real_backend#AerSimulator.from_backend(real_backend)

In [2]:
n = 32  # for a local backend n can go as up as 23, after that it raises a Memory Error
qr = QuantumRegister(n, name='qr')
cr = ClassicalRegister(n, name='cr')
Alice = User(
    name = "Alice",
    key = "",
    basis = [],
    qc = QuantumCircuit(qr, cr, name="a"),
    qr = qr,
    cr = cr,
    n = n,
)
Bob = User(
    name = "Bob",
    key = "",
    basis = [],
    qc = QuantumCircuit(qr, cr, name="b"),
    qr = qr,
    cr = cr,
    n = n,
)
Eve = User(
    name = "Eve",
    key = "",
    basis = [],
    qc = QuantumCircuit(qr, cr, name="c"),
    qr = qr,
    cr = cr,
    n = n,
)

In [3]:
Alice.key = int.from_bytes(randbytes(int(n/8)))
# Cast key to binary for encoding
# range: key[0]-key[15] with key[15] least significant figure
Alice.key = np.binary_repr(Alice.key, n) # n is the width

# Encode key as alice qubits 
# IBM's qubits are all set to |0> initially
for index, digit in enumerate(Alice.key):
    if digit == '1':
        Alice.qc.x(Alice.qr[index]) # if key has a '1', change state to |1>

create_basis(Alice)
create_basis(Bob)
create_basis(Eve)

In [4]:
alice_eve = send_state(Alice, Eve)
counts = execute(alice_eve, backend)

Eve.key = list(counts)[0][::-1]

new_alice_key, new_eve_key = get_shared_key(Alice, Eve)

Percentage of qubits to be discarded according to table comparison:  0.59375
Measurement convergence by additional chance:  0.71875
Percentage of similarity between the keys:  1.0
Key exchange has been successfull
New Alice's key:  0000011000001001100
New Eve's key:  0000011000001001100


In [5]:
eve_bob = send_state(Eve, Bob)
counts = execute(eve_bob, backend)

Bob.key = list(counts)[0][::-1]
get_shared_key(Alice, Bob)

Percentage of qubits to be discarded according to table comparison:  0.53125
Measurement convergence by additional chance:  0.71875
Percentage of similarity between the keys:  0.7058823529411765
Key exchange has been tampered! Check for eavesdropper or try again
New Alice's key is invalid:  00000110000101101
New Bob's key is invalid:  00000101000001000


(['0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '1',
  '1',
  '0',
  '1'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0'])